Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print("train set with size " + str(train_size) +" : ", train_text[:64])
print("valid set" , valid_size,": ", valid_text[:64])

train set with size 99999000 :  ons anarchists advocate social relations based upon voluntary as
valid set 1000 :   anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '


print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [6]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    print("Begining of text: " , text[:64])
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size # // : floor division -> segment = number of batches
    self._cursor = [ offset * segment for offset in range(batch_size)] # indices of first element of a batch
    print("Initial cursor: " , self._cursor)
    self._last_batch = self._next_batch(verbose = True)
  
  def _next_batch(self, verbose = False):
    """Generate a single batch from the current cursor position in the data."""
    # Each position in a batch follows a letter at the same position in the previous batch...
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      if verbose:
        print("batch ", b, ": text[", self._cursor[b],"]= " , self._text[self._cursor[b]])
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    # last_batch is the last character for every batch.
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 2)

print("\nExample Train batch1:" , batches2string(train_batches.next()))
print("\nExample Train batch2:", batches2string(train_batches.next()))
print("\nExample Valid batch1:", batches2string(valid_batches.next()))
print("\nExample Valid batch2:" , batches2string(valid_batches.next()))

Begining of text:  ons anarchists advocate social relations based upon voluntary as
Initial cursor:  [0, 1562484, 3124968, 4687452, 6249936, 7812420, 9374904, 10937388, 12499872, 14062356, 15624840, 17187324, 18749808, 20312292, 21874776, 23437260, 24999744, 26562228, 28124712, 29687196, 31249680, 32812164, 34374648, 35937132, 37499616, 39062100, 40624584, 42187068, 43749552, 45312036, 46874520, 48437004, 49999488, 51561972, 53124456, 54686940, 56249424, 57811908, 59374392, 60936876, 62499360, 64061844, 65624328, 67186812, 68749296, 70311780, 71874264, 73436748, 74999232, 76561716, 78124200, 79686684, 81249168, 82811652, 84374136, 85936620, 87499104, 89061588, 90624072, 92186556, 93749040, 95311524, 96874008, 98436492]
batch  0 : text[ 0 ]=  o
batch  1 : text[ 1562484 ]=  w
batch  2 : text[ 3124968 ]=  l
batch  3 : text[ 4687452 ]=   
batch  4 : text[ 6249936 ]=  m
batch  5 : text[ 7812420 ]=  h
batch  6 : text[ 9374904 ]=  y
batch  7 : text[ 10937388 ]=  a
batch  8 : text[ 12499872 ]=

In [7]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [8]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1): # num_nrollings : length of a single sequence ( there is batch_number of sequences in a batch.)
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [9]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.298181 learning rate: 10.000000
Minibatch perplexity: 27.06
wctwdee  iznbek ro ja fptbom oe eemerhmrbuioaeti nw  dbgrfuk ke wzae ctg ri rago
h  ypnel f iozsnkyka y  q  lo akli ebkzwqam on  tawoyn mmtncp s w rttd mafdwtut 
kul l fzdph e rfac yeekvbgsu r  etbt famhejjemlxts   nice tpkocpe h ea g ldnibpg
ctuc etah dha k zrgtbisllyeziqxbre nniy iox efota  ahuepqqp  ilipcyeqpq gnutsat 
wezmge daih qpgmeebcsyftapzqigb  vowmadst  daltnipj t   eagsm pqle qxeieajauglee
Validation set perplexity: 20.83
Average loss at step 100: 2.581827 learning rate: 10.000000
Minibatch perplexity: 10.95
Validation set perplexity: 13.85
Average loss at step 200: 2.240139 learning rate: 10.000000
Minibatch perplexity: 8.52
Validation set perplexity: 14.90
Average loss at step 300: 2.098517 learning rate: 10.000000
Minibatch perplexity: 7.44
Validation set perplexity: 15.53
Average loss at step 400: 1.998437 learning rate: 10.000000
Minibatch perplexity: 7.64
Validation set p

Validation set perplexity: 25.32
Average loss at step 4500: 1.618500 learning rate: 10.000000
Minibatch perplexity: 5.28
Validation set perplexity: 25.73
Average loss at step 4600: 1.614781 learning rate: 10.000000
Minibatch perplexity: 5.07
Validation set perplexity: 26.44
Average loss at step 4700: 1.628706 learning rate: 10.000000
Minibatch perplexity: 5.23
Validation set perplexity: 26.26
Average loss at step 4800: 1.635516 learning rate: 10.000000
Minibatch perplexity: 4.45
Validation set perplexity: 24.97
Average loss at step 4900: 1.636459 learning rate: 10.000000
Minibatch perplexity: 5.12
Validation set perplexity: 26.08
Average loss at step 5000: 1.607156 learning rate: 1.000000
Minibatch perplexity: 4.52
za by the quicier that penal done as a part shik piminatic dried dess is the rom
vies imposer writer modessial frin lo and sesidn of musts pagess formed but demo
or is the b is d metures the basterms dent they cities of by not by bonitive and
ked with later forgarian exases 

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [10]:
graph2 = tf.Graph()
with graph2.as_default():
  
  # Parameters:
  X  = tf.Variable(tf.truncated_normal([vocabulary_size, 4*num_nodes], -0.1, 0.1)) # multiplies input
  M = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1)) # multiplies output
  B = tf.Variable(tf.zeros([1, 4*num_nodes])) # biases
 
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    
    input_factors = tf.matmul(i, X)
    output_factors = tf.matmul(o, M)
    combined_vector = input_factors + output_factors + B # four calculated values combined (IG, FG, UPDATE, OUTPUT)
    #print("Shapes:","\ninput_factors: " , input_factors.shape,"\n output_factors: " , output_factors.shape,
    #      "\n combined_vector: " , combined_vector.shape)
    
    input_gate = tf.sigmoid(combined_vector[:,:num_nodes])
    forget_gate = tf.sigmoid(combined_vector[:,num_nodes:2*num_nodes])
    update = combined_vector[:,2*num_nodes:3*num_nodes]
    #print("shapes:\n","IG: ", input_gate.shape, "\n FG: " , forget_gate.shape, "\n update: " , update.shape)
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(combined_vector[:,3*num_nodes:])
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1): # num_nrollings : length of a single sequence ( there is batch_number of sequences in a batch.)
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))
    
    

with tf.Session(graph=graph2) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.295103 learning rate: 10.000000
Minibatch perplexity: 26.98
uoeinhme  e jb etlwpdohidzrcotst  e mplzgsbednv roym ofbear  b qiophdsnnf f  dtn
yqeeai a zrde ykatumxatzsagtpnaj rn hvs y  neyfskuyrnlvbnylm t iyhtvioyrlhh emor
f  jkaxersdedhvixsuhxn mncko swherxace lblhqloaco sgzyy pwcvafpo eep  gj  weovi 
qrdwmeiifxgeg mr w zmendubrzlvt zonxp aen h ih  ywvvurbtlogedeiumedeo eje  mka k
yjae demfweorteorarn  i czziflk tcvitresw r ulh inhjp asszqahoas sg waslhzgh tsa
Validation set perplexity: 20.71
Average loss at step 100: 2.579773 learning rate: 10.000000
Minibatch perplexity: 10.80
Validation set perplexity: 14.30
Average loss at step 200: 2.240238 learning rate: 10.000000
Minibatch perplexity: 8.28
Validation set perplexity: 15.68
Average loss at step 300: 2.086463 learning rate: 10.000000
Minibatch perplexity: 6.31
Validation set perplexity: 16.15
Average loss at step 400: 2.037263 learning rate: 10.000000
Minibatch perplexity: 7.95
Validation set p

Validation set perplexity: 25.87
Average loss at step 4500: 1.638821 learning rate: 10.000000
Minibatch perplexity: 5.04
Validation set perplexity: 25.27
Average loss at step 4600: 1.623001 learning rate: 10.000000
Minibatch perplexity: 5.53
Validation set perplexity: 25.13
Average loss at step 4700: 1.614960 learning rate: 10.000000
Minibatch perplexity: 4.72
Validation set perplexity: 25.80
Average loss at step 4800: 1.604306 learning rate: 10.000000
Minibatch perplexity: 4.63
Validation set perplexity: 26.55
Average loss at step 4900: 1.616233 learning rate: 10.000000
Minibatch perplexity: 5.25
Validation set perplexity: 24.74
Average loss at step 5000: 1.613452 learning rate: 1.000000
Minibatch perplexity: 4.81
rately france secture considered for collection weich accented speciqoss in the 
fully were stiressits him has naver guno shost iislate a celdanest createls toke
x charejumension bloeting germ to the bcod servicia funche drom is a diswoint th
quy of as mankst emparing vet on

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

In [11]:
#a: Introducing Embeddings

embedding_size = 10
print("Vocabulary size: " ,vocabulary_size,"\nembedding_size: " , embedding_size  )
graph3 = tf.Graph()
with graph3.as_default():
  
  # Parameters:
  X  = tf.Variable(tf.truncated_normal([embedding_size, 4*num_nodes], -0.1, 0.1)) # multiplies input
  M = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1)) # multiplies output
  B = tf.Variable(tf.zeros([1, 4*num_nodes])) # biases
 
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
    
  #for embeddings:
  embeddings = tf.Variable(
  tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))

  
  # Definition of the cell computation.
  def lstm_cell(i_embeddings, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_factors = tf.matmul(i_embeddings, X)
    output_factors = tf.matmul(o, M)
    combined_vector = input_factors + output_factors + B # four calculated values combined (IG, FG, UPDATE, OUTPUT)
    #print("Shapes:","\ninput_factors: " , input_factors.shape,"\n output_factors: " , output_factors.shape,
    #      "\n combined_vector: " , combined_vector.shape)
    
    input_gate = tf.sigmoid(combined_vector[:,:num_nodes])
    forget_gate = tf.sigmoid(combined_vector[:,num_nodes:2*num_nodes])
    update = combined_vector[:,2*num_nodes:3*num_nodes]
    #print("shapes:\n","IG: ", input_gate.shape, "\n FG: " , forget_gate.shape, "\n update: " , update.shape)
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(combined_vector[:,3*num_nodes:])
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1): # num_nrollings : length of a single sequence ( there is batch_number of sequences in a batch.)
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

    

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  #converting input onthot to character number:
  for input in train_inputs:
    index = tf.to_int32(tf.matmul(input, np.array([range(vocabulary_size)], 
        dtype=np.float32).T))
    
    
    train_embed_inputs = tf.squeeze(tf.nn.embedding_lookup(embeddings, index))
 
    output, state = lstm_cell(train_embed_inputs, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  index = tf.to_int32(tf.matmul(sample_input, np.array([range(vocabulary_size)], 
        dtype=np.float32).T))
  sample_output, sample_state = lstm_cell(
    tf.squeeze(tf.nn.embedding_lookup(embeddings, index),1), saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))
    
    

with tf.Session(graph=graph3) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Vocabulary size:  27 
embedding_size:  10
Initialized
Average loss at step 0: 3.298410 learning rate: 10.000000
Minibatch perplexity: 27.07
aiu ejdcttskogyonlkclmron  g  ughicilnjiauztbc hd  xyemadicgco eeyvlilcyhyasy em
ldhotdehed vkzishosfiw gwi yotkwsegy vtj strgal  xeu cl e  wkvesn eivt wpvtfoezp
u ahk sbyes il ee nqojzau itywehqeewlawtep eatidetzf  tiayej  o r i  qirmbetjoss
joouundeeuawfvr  z nnoiplnvyuofaaqccofnerm y hemzubbyute so xynzrh etql ehfukm x
 tcc ha aanaf y w koe o ah btugslowb nmmyjs   te ic mt iyiaobwtcpxywdqr tyoca zm
Validation set perplexity: 20.63
Average loss at step 100: 2.434775 learning rate: 10.000000
Minibatch perplexity: 11.09
Validation set perplexity: 14.45
Average loss at step 200: 2.115241 learning rate: 10.000000
Minibatch perplexity: 7.50
Validation set perplexity: 18.12
Average loss at step 300: 1.977445 learning rate: 10.000000
Minibatch perplexity: 6.31
Validation set perplexity: 19.05
Average loss at step 400: 1.905529 learning rate: 10.000000
M

Average loss at step 4400: 1.664448 learning rate: 10.000000
Minibatch perplexity: 5.31
Validation set perplexity: 27.68
Average loss at step 4500: 1.672081 learning rate: 10.000000
Minibatch perplexity: 5.44
Validation set perplexity: 27.32
Average loss at step 4600: 1.670626 learning rate: 10.000000
Minibatch perplexity: 5.62
Validation set perplexity: 27.77
Average loss at step 4700: 1.645491 learning rate: 10.000000
Minibatch perplexity: 5.71
Validation set perplexity: 28.73
Average loss at step 4800: 1.626619 learning rate: 10.000000
Minibatch perplexity: 5.55
Validation set perplexity: 29.66
Average loss at step 4900: 1.644822 learning rate: 10.000000
Minibatch perplexity: 5.29
Validation set perplexity: 29.14
Average loss at step 5000: 1.670851 learning rate: 1.000000
Minibatch perplexity: 5.55
t lox clui threen but mickotific lord dease surgotoball hydrowing franned also t
w infrominis the social are justilan alto mps oten the instillier when ob maniet
wer d a gebral das to seb

In [12]:
#b: Introducing bigrams
valid_batches = BatchGenerator(valid_text, 1, 2)

bigram_voc_size =vocabulary_size * vocabulary_size

embedding_size = 10
print("Vocabulary size: " ,vocabulary_size,"\nembedding_size: " , embedding_size  )

graph4 = tf.Graph()
with graph4.as_default():
  
  # Parameters:
  X  = tf.Variable(tf.truncated_normal([embedding_size, 4*num_nodes], -0.1, 0.1)) # multiplies input
  M = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1)) # multiplies output
  B = tf.Variable(tf.zeros([1, 4*num_nodes])) # biases
 
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
    
  #for embeddings:
  embeddings = tf.Variable(
  tf.random_uniform([vocabulary_size * vocabulary_size, embedding_size], -1.0, 1.0)) # increased size to store all combinations

  
  # Definition of the cell computation.
  def lstm_cell(i_embeddings, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_factors = tf.matmul(i_embeddings, X)
    output_factors = tf.matmul(o, M)
    combined_vector = input_factors + output_factors + B # four calculated values combined (IG, FG, UPDATE, OUTPUT)
    #print("Shapes:","\ninput_factors: " , input_factors.shape,"\n output_factors: " , output_factors.shape,
    #      "\n combined_vector: " , combined_vector.shape)
    
    input_gate = tf.sigmoid(combined_vector[:,:num_nodes])
    forget_gate = tf.sigmoid(combined_vector[:,num_nodes:2*num_nodes])
    update = combined_vector[:,2*num_nodes:3*num_nodes]
    #print("shapes:\n","IG: ", input_gate.shape, "\n FG: " , forget_gate.shape, "\n update: " , update.shape)
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(combined_vector[:,3*num_nodes:])
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1): # num_nrollings : length of a single sequence ( there is batch_number of sequences in a batch.)
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.
  train_labels.pop(0) # removes first element: predicting on bigrams, so I can not predict the first letter.

    

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  #converting input onthot to character number:
  for i in range(len(train_inputs)-1):
    firstPos = i
    secondPos = i+1
    firstIndex = tf.to_int32(tf.matmul(train_inputs[firstPos], np.array([range(vocabulary_size)], 
        dtype=np.float32).T))
    secondIndex = tf.to_int32(tf.matmul(train_inputs[secondPos], np.array([range(vocabulary_size)], 
        dtype=np.float32).T))
    
    
    train_embed_inputs = tf.squeeze(tf.nn.embedding_lookup(embeddings, firstIndex * vocabulary_size +secondIndex ))
 
    output, state = lstm_cell(train_embed_inputs, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[2,1, vocabulary_size]) # extended to 2 letters
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  index1 = tf.to_int32(tf.matmul(sample_input[0], np.array([range(vocabulary_size)], 
        dtype=np.float32).T))
  index2 = tf.to_int32(tf.matmul(sample_input[1], np.array([range(vocabulary_size)], 
        dtype=np.float32).T))
  sample_output, sample_state = lstm_cell(
    tf.squeeze(tf.nn.embedding_lookup(embeddings, index1 * vocabulary_size + index2),1), saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))
    
    

with tf.Session(graph=graph4) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[2:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed1 = sample(random_distribution())
          firstLetter = characters(feed1)[0]
          feed2 = sample(random_distribution())
          secondLetter = characters(feed2)[0]
          sentence = firstLetter + secondLetter
          feed = np.array([feed1, feed2])


          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            predicted = sample(prediction)
            sentence += characters(predicted)[0]
            feed[0] = feed[1]
            feed[1] = predicted
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: np.array([b[0],b[1]])})
        valid_logprob = valid_logprob + logprob(predictions, b[2])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Begining of text:   anarchism originated as a term of abuse first used against earl
Initial cursor:  [0]
batch  0 : text[ 0 ]=   
Vocabulary size:  27 
embedding_size:  10
Initialized
Average loss at step 0: 3.300901 learning rate: 10.000000
Minibatch perplexity: 27.14
ursjbtguetgt hoaeau bqlwz vukwpdgoxnog tnesmwnea de cvr rbnueluamrpiteeeit b tlta
smq mevsicitrp hhpv lp cnygptidoa icn bikfeaanm dnxtsi  xtn g kjlcswisf spmmlrcjy
jytg  tkxiofufayq exftqsp aasceex pxourppq  naq wiojqejxxxi or upjevt pf rok mlwe
kgktkhbkxgltrvlsqv c utul ieoinqpdsfa hio tnhalipeuiwlixi ij ufibalnjbst sa ssrfp
m inowmjuoootwewcsnmbbl dwen yhnpd eqlqyz fhhnseomzgibrt vr mehorrtdtbstanicc j s
Validation set perplexity: 19.84
Average loss at step 100: 2.605200 learning rate: 10.000000
Minibatch perplexity: 10.19
Validation set perplexity: 10.85
Average loss at step 200: 2.230040 learning rate: 10.000000
Minibatch perplexity: 8.41
Validation set perplexity: 9.22
Average loss at step 300: 2.085725 learning rat

Average loss at step 4300: 1.698827 learning rate: 10.000000
Minibatch perplexity: 6.43
Validation set perplexity: 8.43
Average loss at step 4400: 1.729066 learning rate: 10.000000
Minibatch perplexity: 5.75
Validation set perplexity: 8.15
Average loss at step 4500: 1.719222 learning rate: 10.000000
Minibatch perplexity: 5.80
Validation set perplexity: 8.44
Average loss at step 4600: 1.723181 learning rate: 10.000000
Minibatch perplexity: 5.47
Validation set perplexity: 7.95
Average loss at step 4700: 1.740696 learning rate: 10.000000
Minibatch perplexity: 5.60
Validation set perplexity: 8.29
Average loss at step 4800: 1.730364 learning rate: 10.000000
Minibatch perplexity: 5.08
Validation set perplexity: 8.17
Average loss at step 4900: 1.757776 learning rate: 10.000000
Minibatch perplexity: 6.01
Validation set perplexity: 8.40
Average loss at step 5000: 1.759132 learning rate: 1.000000
Minibatch perplexity: 5.41
ic miet of stonesics formt when bez rety by the scompoling x four three a

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---